In [25]:
#equal number in each slice for SIR
from tabulate import tabulate
import numpy as np
from scipy.linalg import solve_triangular
def sir_modified(X, y, num_slices, K):
    #
    X = X - np.mean(X, axis=0)
    Q, R = np.linalg.qr(X)
    n_samples, n_features = X.shape
    Z = np.sqrt(n_samples) * Q
    
    V_hat = np.zeros([X.shape[1], X.shape[1]])
    # Step 1: Sort the data by the response variable
    sorted_indices = np.argsort(y)
    Z_sorted = Z[sorted_indices]
    y_sorted = y[sorted_indices]

    # Step 2: Divide the data into slices
    slice_size = n_samples // num_slices
    ph_hat = slice_size/n_samples
    slices = []

    for i in range(num_slices):
        start_idx = i * slice_size
        if i < num_slices - 1:
            end_idx = (i + 1) * slice_size
        else:  # Last slice includes any remaining samples
            end_idx = n_samples
        slices.append((Z_sorted[start_idx:end_idx], y_sorted[start_idx:end_idx]))

    # Step 3: Compute the means of the predictors within each slice
    X_means = np.array([np.mean(slice_X, axis=0) for slice_X, _ in slices])

    # Step 4: Center the predictor means
    X_centered = X_means - np.mean(X_means, axis=0)

    V_hat = np.add(V_hat,ph_hat * np.matmul(X_centered.T, X_centered))
    
    eigenvalues, eigenvectors = np.linalg.eig(V_hat)
    
    idx = eigenvalues.argsort()[::-1]  # Get indices that would sort eigenvalues in descending order
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    edr_est = solve_triangular(np.sqrt(n_samples) * R, eigenvectors)
    K_index = np.argpartition(np.abs(eigenvalues), X.shape[1]-K) >= X.shape[1]-K
    edr_est = edr_est[:, K_index]
    # edr_est = edr_est.flatten()
    if edr_est[0][0] < 0:
        edr_est = -edr_est
    edr_est = edr_est / np.linalg.norm(edr_est)
    
    return edr_est.T, V_hat

In [28]:
#sir_modified test on 2*z1t + 3*z2t
import sliced
from sliced import SlicedInverseRegression
num_N = 5
n_obs = 10000
ar_coeff = [0.2, 0.3, 0.5, 0.8]
noise = np.zeros((num_N, n_obs))
n = 100
H = 50
K = 1
hat = 0
Hat = 0
for a in range(n):
    for h in range(num_N):
        noise[h] = np.random.normal(0, 1, size=n_obs)  # Normally distributed noise
    ar_series = np.zeros((num_N, n_obs))
    for t in range(0, n_obs):
        ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1]  + noise[0][t]
        ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1]  + noise[1][t]
        ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1]  + noise[2][t]
        ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1]  + noise[3][t]
        ar_series[4][t] = 2*ar_series[0][t] + 4*ar_series[1][t] + noise[4][t]
    y = ar_series[4]
    X = np.concatenate([ar_series[i].reshape(-1,1) for i in range(4)], axis = 1)
    edr, _ = sir_modified(X, y, H, K=K)
    hat += edr

hat = hat/n
g = hat[0][0]/hat[0][1]
g

0.5001657901876191

In [14]:
#sir_modified test on 2*z1t + 3*z2t
import sliced
from sliced import SlicedInverseRegression
num_N = 5
n_obs = 10000
ar_coeff = [0.2, 0.3, 0.5, 0.8]
noise = np.zeros((num_N, n_obs))
n = 100
H = 50
K = 1
hat = 0
Hat = 0
for a in range(n):
    for h in range(num_N):
        noise[h] = np.random.normal(0, 1, size=n_obs)  # Normally distributed noise
    ar_series = np.zeros((num_N, n_obs))
    for t in range(0, n_obs):
        ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1]  + noise[0][t]
        ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1]  + noise[1][t]
        ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1]  + noise[2][t]
        ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1]  + noise[3][t]
        ar_series[4][t] = 3*ar_series[0][t] + 4*ar_series[1][t] + noise[4][t]
    y = ar_series[4]
    X = np.concatenate([ar_series[i].reshape(-1,1) for i in range(4)], axis = 1)
    
    sir = SlicedInverseRegression(n_directions=1)
    
    sir.fit(X, y)
    if sir.directions_[0][0] < 0:
        sir.directions_ = -sir.directions_
    hat += sir.directions_

hat = hat/n
g = hat[0][0]/hat[0][1]
g

0.7496967032855786

In [11]:
#sir package on 2*z1(t-1) + 3*z2(t-1): TSIR's model
import sliced
from sliced import SlicedInverseRegression
num_N = 5
n_obs = 10000
ar_coeff1 = 0.2
ar_coeff2 = 0.8
noise = np.zeros((num_N, n_obs))
n = 100
H = 50
K = 1
hat = 0
Hat = 0
for a in range(n):
    for h in range(num_N):
        noise[h] = np.random.normal(0, 1, size=n_obs)  # Normally distributed noise
    ar_series = np.zeros((num_N, n_obs))
    for t in range(0, n_obs):
        ar_series[0][t] = ar_coeff1 * ar_series[0][t - 1]  + noise[0][t]
        ar_series[1][t] = ar_coeff2 * ar_series[1][t - 1]  + noise[1][t]
        ar_series[2][t] = 0.3 * ar_series[2][t - 1] + 0.4 * noise[2][t-1] + noise[2][t]
        ar_series[3][t] = -0.4 * noise[3][t-1] + noise[3][t]
        ar_series[4][t] = 2*ar_series[0][t - 1] + 3*ar_series[1][t - 1] + noise[4][t]
    y = ar_series[4]
    X = np.concatenate([ar_series[i].reshape(-1,1) for i in range(4)], axis = 1)
    
    sir = SlicedInverseRegression(n_directions=1)
    
    sir.fit(X, y)
    if sir.directions_[0][0] < 0:
        sir.directions_ = -sir.directions_
    hat += sir.directions_

hat = hat/n
g = hat[0][0]/hat[0][1]
g

0.16616637671152745

In [30]:
#from 0 to Q double sum
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
    
def test(ar_coeff):
    import numpy as np
    from tabulate import tabulate
    num_N = 5
    n_obs = 10000
    S = 20
    noise = np.zeros((num_N, n_obs+S))
    n = 100
    H = 50
    P = 4
    K = 1
    y = [np.zeros((num_N, n_obs+i)) for i in range(S+1)]
    hat = [np.zeros((P, 1)) for _ in range(S)]
    g = np.zeros((S, 1))
    n1 = 0
    l = 1  # Initialize `l` outside the loop
    while n1 < 100:
        for h in range(num_N):
            noise[h] = np.random.normal(0, 1, size=(n_obs+S))  # Normally distributed noise
        ar_series = np.zeros((num_N, n_obs+S))
        for t in range(0, n_obs+S):
            ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
            ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
            ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
            ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
            ar_series[4][t] = 2 * ar_series[0][t] + 3 * ar_series[1][t] + noise[4][t]
        for a in range(0, S+1):
            y[a] = ar_series[4][a:n_obs+a]
        X = np.concatenate([ar_series[i][0:n_obs].reshape(-1, 1) for i in range(4)], axis=1)
        V = []
        for a in range(0, S + 1):
            _ , M = sir_modified(X, y[a], H, K)
            V.append(M)
        for q in range(1, S + 1):
            Q = np.zeros((P, P))
            phi = ar_coeff
            for j in range(P):
                for k in range(P):
                    Q[j, k] = sum(sum(phi[j] ** a * V[a][j, k] * phi[k] ** a for a in range(0, l)) for l in range(1, q + 1))
            eigenvalues1, eigenvectors1 = np.linalg.eig(Q)
            K_index = np.argpartition(np.abs(eigenvalues1), P - K) >= P - K
            K_largest_eigenvectors = eigenvectors1[:, K_index]
            edr_est = K_largest_eigenvectors
            if edr_est[0] < 0:
                edr_est = -edr_est
            edr_est = edr_est / np.linalg.norm(edr_est)
            hat[q - 1] += edr_est
            n1 += 1
    
    for i in range(S):
        hat[i] = hat[i] / n
        g[i] = hat[i][0] / hat[i][1]
    array = np.array(hat)
    print(tabulate(array, tablefmt='latex'))
    print(g)

# Example usage
ar_coeff = [0.2, 0.2, 0.2, 0.2]
test(ar_coeff)


\begin{tabular}{rrrr}
\hline
 0.0279931 & 0.00805542 & 3.82269e-06 & 8.43091e-05 \\
 0.0279921 & 0.00805542 & 4.6094e-06  & 8.35304e-05 \\
 0.0279917 & 0.00805542 & 4.854e-06   & 8.32811e-05 \\
 0.0279916 & 0.00805541 & 4.97621e-06 & 8.31564e-05 \\
 0.0279915 & 0.00805541 & 5.04952e-06 & 8.30816e-05 \\
 0.0279914 & 0.00805541 & 5.09838e-06 & 8.30318e-05 \\
 0.0279914 & 0.00805541 & 5.13328e-06 & 8.29962e-05 \\
 0.0279913 & 0.00805541 & 5.15945e-06 & 8.29695e-05 \\
 0.0279913 & 0.00805541 & 5.17981e-06 & 8.29487e-05 \\
 0.0279913 & 0.00805541 & 5.19609e-06 & 8.29321e-05 \\
 0.0279913 & 0.00805541 & 5.20941e-06 & 8.29185e-05 \\
 0.0279912 & 0.00805541 & 5.22051e-06 & 8.29072e-05 \\
 0.0279912 & 0.00805541 & 5.22991e-06 & 8.28976e-05 \\
 0.0279912 & 0.00805541 & 5.23796e-06 & 8.28894e-05 \\
 0.0279912 & 0.00805541 & 5.24494e-06 & 8.28822e-05 \\
 0.0279912 & 0.00805541 & 5.25104e-06 & 8.2876e-05  \\
 0.0279912 & 0.00805541 & 5.25643e-06 & 8.28705e-05 \\
 0.0279912 & 0.00805541 & 5.26122e-0